In [1]:
import cv2 as cv
import numpy as np

MAX_HUMANS_TO_TRACK = 10

class KalmanFilter:
    def __init__(self):
        self.kf = cv.KalmanFilter(4, 2)
        self.kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
        self.kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

    def Estimate(self, coordX, coordY):
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        predicted = self.kf.predict()
        self.kf.correct(measured)
        return predicted

class ProcessImage:
    def __init__(self):
        self.human_cascade = cv.CascadeClassifier('haarcascade_fullbody.xml') # Path to Haar cascade for human detection
        self.kfObjs = []
        for _ in range(MAX_HUMANS_TO_TRACK):
            self.kfObjs.append(KalmanFilter())
        self.frame_count = 0
        self.detector_frequency = 1  # Perform detection every 5 frames

    def DetectHumans(self, frame):
        self.frame_count += 1
        if self.frame_count % self.detector_frequency == 0:
            gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
            humans = self.human_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
            return humans
        else:
            return []

    def PredictMotion(self, frame, humans):
        for i, human in enumerate(humans):
            if i >= MAX_HUMANS_TO_TRACK:
                break
            x, y, w, h = human
            human_center = (x + w // 2, y + h // 2)
            predicted_coords = self.kfObjs[i].Estimate(*human_center)
            # Draw the actual position
            cv.circle(frame, human_center, 20, [0, 0, 255], 2, 8)
            cv.putText(frame, "Actual", (human_center[0] + 50, human_center[1] + 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, [0, 0, 255], 2)
            # Draw the predicted position
            cv.circle(frame, (int(predicted_coords[0]), int(predicted_coords[1])), 20, [0, 255, 255], 2, 8)
            cv.putText(frame, "Predicted", (int(predicted_coords[0] + 50), int(predicted_coords[1] + 20)), cv.FONT_HERSHEY_SIMPLEX, 0.5, [0, 255, 255], 2)
        return frame

def main():
    processImg = ProcessImage()
    vid = cv.VideoCapture('green.mp4')
    if not vid.isOpened():
        print('Cannot open input video')
        return

    while True:
        ret, frame = vid.read()
        if not ret:
            break

        humans = processImg.DetectHumans(frame)
        frame = processImg.PredictMotion(frame, humans)

        cv.imshow('Human Detection', frame)
        if cv.waitKey(30) & 0xFF == ord('q'):
            break

    vid.release()
    cv.destroyAllWindows()

if __name__ == "__main__":
    main()

print('Program Completed!')


/tmp/ipykernel_12628/1239395359.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv.circle(frame, (int(predicted_coords[0]), int(predicted_coords[1])), 20, [0, 255, 255], 2, 8)
/tmp/ipykernel_12628/1239395359.py:48: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cv.putText(frame, "Predicted", (int(predicted_coords[0] + 50), int(predicted_coords[1] + 20)), cv.FONT_HERSHEY_SIMPLEX, 0.5, [0, 255, 255], 2)


KeyboardInterrupt: 

In [ ]:
import cv2 as cv
import numpy as np

MAX_HUMANS_TO_TRACK = 20

class KalmanFilter:
    def __init__(self):
        self.kf = cv.KalmanFilter(4, 2)
        self.kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
        self.kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

    def Estimate(self, coordX, coordY):
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        predicted = self.kf.predict()
        self.kf.correct(measured)
        return predicted

class ProcessImage:
    def __init__(self):
        self.net = cv.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')  # Load YOLOv3 model
        self.output_layers = self.net.getUnconnectedOutLayersNames()
        self.kfObjs = []
        for _ in range(MAX_HUMANS_TO_TRACK):
            self.kfObjs.append(KalmanFilter())
        self.frame_count = 0
        self.detector_frequency = 1  # Perform detection every frame

    def DetectHumans(self, frame):
        self.frame_count += 1
        blob = cv.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == 0:  # Class ID for person in COCO dataset
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    w = int(detection[2] * frame.shape[1])
                    h = int(detection[3] * frame.shape[0])
                    x = center_x - w // 2
                    y = center_y - h // 2
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
        indexes = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        humans = []
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                humans.append((x, y, w, h))
        return humans

    def PredictMotion(self, frame, humans):
        for i, human in enumerate(humans):
            if i >= MAX_HUMANS_TO_TRACK:
                break
            x, y, w, h = human
            human_center = (x + w // 2, y + h // 2)
            predicted_coords = self.kfObjs[i].Estimate(*human_center)
            # Draw the actual position
            cv.circle(frame, human_center, 20, [0, 0, 255], 2, 8)
            cv.putText(frame, "Actual", (human_center[0] + 50, human_center[1] + 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, [0, 0, 255], 2)
            # Draw the predicted position
            cv.circle(frame, (int(predicted_coords[0]), int(predicted_coords[1])), 20, [0, 255, 255], 2, 8)
            cv.putText(frame, "Predicted", (int(predicted_coords[0] + 50), int(predicted_coords[1] + 20)), cv.FONT_HERSHEY_SIMPLEX, 0.5, [0, 255, 255], 2)
        return frame

def main():
    processImg = ProcessImage()
    vid = cv.VideoCapture('crowd.mp4')
    if not vid.isOpened():
        print('Cannot open input video')
        return

    while True:
        ret, frame = vid.read()
        if not ret:
            break

        humans = processImg.DetectHumans(frame)
        frame = processImg.PredictMotion(frame, humans)

        cv.imshow('Human Detection', frame)
        if cv.waitKey(30) & 0xFF == ord('q'):
            break

    vid.release()
    cv.destroyAllWindows()

if __name__ == "__main__":
    main()

print('Program Completed!')


In [1]:
import cv2
import numpy as np

MAX_HUMANS_TO_TRACK = 20

class KalmanFilter:
    def __init__(self):
        self.kf = cv2.KalmanFilter(4, 2)
        self.kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
        self.kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

    def Estimate(self, coordX, coordY):
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        predicted = self.kf.predict()
        self.kf.correct(measured)
        return predicted

class ProcessImage:
    def __init__(self):
        self.net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')  # Load YOLOv3 model
        self.output_layers = self.net.getUnconnectedOutLayersNames()
        self.kfObjs = []
        self.prev_centers = {}
        for _ in range(MAX_HUMANS_TO_TRACK):
            self.kfObjs.append(KalmanFilter())
        self.frame_count = 0
        self.detector_frequency = 1  # Perform detection every frame

    def DetectHumans(self, frame):
        self.frame_count += 1
        blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
        self.net.setInput(blob)
        outs = self.net.forward(self.output_layers)
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5 and class_id == 0:  # Class ID for person in COCO dataset
                    center_x = int(detection[0] * frame.shape[1])
                    center_y = int(detection[1] * frame.shape[0])
                    w = int(detection[2] * frame.shape[1])
                    h = int(detection[3] * frame.shape[0])
                    x = center_x - w // 2
                    y = center_y - h // 2
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
        humans = []
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                center = (x + w // 2, y + h // 2)
                humans.append((center, (x, y, w, h)))
        return humans

    def PredictMotion(self, frame, humans):
        for i, (center, box) in enumerate(humans):
            if i >= MAX_HUMANS_TO_TRACK:
                break
            prev_center = self.prev_centers.get(i)
            if prev_center:
                prev_center = tuple(map(int, prev_center))
                predicted_coords = self.kfObjs[i].Estimate(*prev_center)
                predicted_center = (int(predicted_coords[0]), int(predicted_coords[1]))
                cv2.circle(frame, predicted_center, 20, [0, 255, 255], 2, 8)
                cv2.putText(frame, "Predicted", (predicted_center[0] + 50, predicted_center[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 255, 255], 2)
            self.prev_centers[i] = center
            cv2.circle(frame, center, 20, [0, 0, 255], 2, 8)
            cv2.putText(frame, "Actual", (center[0] + 50, center[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 0, 255], 2)
        return frame

def main():
    processImg = ProcessImage()
    vid = cv2.VideoCapture('fast.mp4')
    if not vid.isOpened():
        print('Cannot open input video')
        return

    while True:
        ret, frame = vid.read()
        if not ret:
            break

        humans = processImg.DetectHumans(frame)
        frame = processImg.PredictMotion(frame, humans)

        cv2.imshow('Human Detection', frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    vid.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

print('Program Completed!')


/tmp/ipykernel_12748/3407192093.py:69: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_center = (int(predicted_coords[0]), int(predicted_coords[1]))


KeyboardInterrupt: 

In [ ]:
import cv2
import numpy as np

MAX_HUMANS_TO_TRACK = 20

class KalmanFilter:
    def __init__(self):
        self.kf = cv2.KalmanFilter(4, 2)
        self.kf.measurementMatrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0]], np.float32)
        self.kf.transitionMatrix = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]], np.float32)

    def Estimate(self, coordX, coordY):
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        predicted = self.kf.predict()
        self.kf.correct(measured)
        return predicted

class ProcessImage:
    def __init__(self):
        self.net = cv2.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')  # Load YOLOv3 model
        self.output_layers = self.net.getUnconnectedOutLayersNames()
        self.kfObjs = []
        self.humans = []
        self.prev_centers = {}
        for _ in range(MAX_HUMANS_TO_TRACK):
            self.kfObjs.append(KalmanFilter())
        self.frame_count = 0
        self.detector_frequency = 30  # Perform detection every 30 frames
        self.detected_once = False

    def DetectHumans(self, frame):
        if not self.detected_once or self.frame_count % self.detector_frequency == 0:
            blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), swapRB=True, crop=False)
            self.net.setInput(blob)
            outs = self.net.forward(self.output_layers)
            class_ids = []
            confidences = []
            boxes = []
            for out in outs:
                for detection in out:
                    scores = detection[5:]
                    class_id = np.argmax(scores)
                    confidence = scores[class_id]
                    if confidence > 0.5 and class_id == 0:  # Class ID for person in COCO dataset
                        center_x = int(detection[0] * frame.shape[1])
                        center_y = int(detection[1] * frame.shape[0])
                        w = int(detection[2] * frame.shape[1])
                        h = int(detection[3] * frame.shape[0])
                        x = center_x - w // 2
                        y = center_y - h // 2
                        boxes.append([x, y, w, h])
                        confidences.append(float(confidence))
                        class_ids.append(class_id)
            indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
            self.humans = []
            for i in range(len(boxes)):
                if i in indexes:
                    x, y, w, h = boxes[i]
                    center = (x + w // 2, y + h // 2)
                    self.humans.append((center, (x, y, w, h)))
            self.detected_once = True
        return self.humans

    def PredictMotion(self, frame):
        for i, (center, box) in enumerate(self.humans):
            if i >= MAX_HUMANS_TO_TRACK:
                break
            prev_center = self.prev_centers.get(i)
            if prev_center:
                prev_center = tuple(map(int, prev_center))
                predicted_coords = self.kfObjs[i].Estimate(*prev_center)
                predicted_center = (int(predicted_coords[0]), int(predicted_coords[1]))
                cv2.circle(frame, predicted_center, 20, [0, 255, 255], 2, 8)
                cv2.putText(frame, "Predicted", (predicted_center[0] + 50, predicted_center[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 255, 255], 2)
            self.prev_centers[i] = center
            cv2.circle(frame, center, 20, [0, 0, 255], 2, 8)
            cv2.putText(frame, "Actual", (center[0] + 50, center[1] + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 0, 255], 2)
        return frame

def main():
    processImg = ProcessImage()
    vid = cv2.VideoCapture('fast.mp4')
    if not vid.isOpened():
        print('Cannot open input video')
        return

    while True:
        ret, frame = vid.read()
        if not ret:
            break

        humans = processImg.DetectHumans(frame)
        frame = processImg.PredictMotion(frame)

        cv2.imshow('Human Detection', frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    vid.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

print('Program Completed!')
